#### P[i] is P(X = i) (sage automatically normalizes). The second cell demonstrates that we do indeed get what we want to. 

In [614]:
U = RealDistribution('uniform', [0, 1])
U.get_random_element()
survival = [1/2, 1/2, 5/6]
#These values for s_i give a fitness of about 1.03 (see birth_distn_experiment)

In [615]:
#A squirrel is a list that looks like [birth_distn, cached_nuts]. A population of squirrels is a list
#    of such lists. 

def single_squirrel_maker(birth_distn, starting_nuts = 0):
    return([birth_distn, starting_nuts])

def many_squirrel_maker(n, birth_distn, starting_nuts = 0):
    return([[birth_distn, starting_nuts] for s in range(n)])

In [616]:
def death_process(squirrels, survival = [1/2, 1/2, 6/7]):
    dead_squirrels = []
    for s in squirrels:
        d = U.get_random_element()
        squirrel_dies = d > survival[s[1]]
        if squirrel_dies:
            dead_squirrels.append(s)
    for dead in dead_squirrels:
        squirrels.remove(dead)
    return(squirrels)

In [617]:
squirrels = many_squirrel_maker(100, [0, 0, 1])
len(death_process(squirrels))

48

In [618]:
def birth_process(squirrels):
    baby_squirrels = []
    for i, s in enumerate(squirrels):
        b = U.get_random_element()
        gives_birth = b < s[0][s[1]]
        if gives_birth:
            baby_squirrels.append(single_squirrel_maker(s[0]))
        else:
            squirrels[i] = single_squirrel_maker(s[0], s[1] + 1)
    squirrels = squirrels + baby_squirrels
    return(squirrels)

In [619]:
squirrels = squirrel_maker(100, [1/2, 0, 1])
len(birth_process(squirrels))

157

In [620]:
def single_day(squirrels, survival = [1/2, 1/2, 5/6]):
    squirrels = death_process(squirrels, survival = survival)
    squirrels = birth_process(squirrels)
    return(squirrels)

In [621]:
def many_days(n, squirrels, survival = [1/2, 1/2, 5/6]):
    for _ in range(n):
        squirrels = death_process(squirrels, survival = survival)
        squirrels = birth_process(squirrels)
    return(len(squirrels))

In [639]:
results = []
squirrels = many_squirrel_maker(10, [1/2, 0, 1], 0)
for _ in range(10000):
    x = many_days(100, squirrels)
    results.append(x)
mean(results).n()

0.0429000000000000

In [638]:
results = []
squirrels = many_squirrel_maker(10, [0, 0, 1], 0)
for _ in range(1000):
    x = many_days(100, squirrels)
    results.append(x)
mean(results).n()

1.25900000000000

#### It looks like we are okay now. These results are an instance of what I was finding in BirthDistnWithMatrices. The next step is to let an evolutionary process guide us toward the optimal birth distribution b. Doing so will be tough. 